In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Profit Loss Calcualtion**

In [4]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/defence_target_data")
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/defence_target_data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder)  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['visualize_Final_data.csv',
 'trade_defence_target.csv',
 'cal_defence_target.csv',
 'ideal_defence_target.csv',
 '.ipynb_checkpoints']

In [8]:
trade_data = pd.read_csv('trade_defence_target.csv')
cal_data = pd.read_csv('cal_defence_target.csv')
ideal_data = pd.read_csv('ideal_defence_target.csv')

In [9]:
print(trade_data.shape)
print(cal_data.shape)
print(ideal_data.shape)

(505221, 24)
(505221, 24)
(505221, 25)


In [ ]:
ideal_data.drop(columns=["Target"],axis=1,inplace=True)
ideal_data.rename(columns={"ideal_target":"Target"},inplace=True)

In [12]:
ideal_data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence',
       'Stop', 'Target', 'ideal_target', 'defence_target'],
      dtype='object')

In [15]:
import pandas as pd
from tqdm import tqdm

def process_data(df):
    # Initialize columns with 0
    df['Buyer_Profit'] = 0
    df['Buyer_Loss'] = 0
    df['Seller_Profit'] = 0
    df['Seller_Loss'] = 0

    # Convert 'Time Stamp' to datetime for efficient comparison

    columns_to_drop = ['Unnamed: 0']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)

    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    # Use a list to collect results for better performance
    results = []

    # Iterate through each row to perform the logic
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing Rows'):
        # Calculate Buyer Profit and Loss
        if row['BuyersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['Target']

            if target_value > entry_value:
                buyer_profit = target_value - entry_value
                buyer_loss = 0
            else:
                buyer_profit = 0
                buyer_loss = abs(target_value - entry_value)
        else:
            buyer_profit = 0
            buyer_loss = 0

        # Calculate Seller Profit and Loss
        if row['SellersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['Target']

            if entry_value > target_value:
                seller_profit = entry_value - target_value
                seller_loss = 0
            else:
                seller_profit = 0
                seller_loss = abs(entry_value - target_value)
        else:
            seller_profit = 0
            seller_loss = 0

        # Append results
        results.append({
            'index': index,
            'Buyer_Profit': buyer_profit,
            'Buyer_Loss': buyer_loss,
            'Seller_Profit': seller_profit,
            'Seller_Loss': seller_loss
        })

    # Update DataFrame with results
    results_df = pd.DataFrame(results)
    df.update(results_df.set_index('index'))
    df['Profit'] = df['Buyer_Profit'] + df['Seller_Profit']
    df['Loss'] = df['Buyer_Loss'] + df['Seller_Loss']
    df.drop(columns=["Buyer_Profit","Buyer_Loss","Seller_Profit","Seller_Loss"],axis=1,inplace=True)


    return df


In [16]:
trade_data_pro_loss = process_data(trade_data)
cal_data_pro_loss = process_data(cal_data)
ideal_data_pro_loss = process_data(ideal_data)

Processing Rows: 100%|██████████| 505221/505221 [00:42<00:00, 11794.23it/s]


In [19]:
trade_data_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/ttrade_data_pro_loss.csv")
cal_data_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/cal_data_pro_loss.csv")
ideal_data_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Target_Optimization_data/Profit_loss_calculation_data/ideal_data_pro_loss.csv")

Calculation For Defence_Profit_Loss


In [22]:
import pandas as pd
from tqdm import tqdm

def defence_process_data(df):
    # Initialize columns with 0
    df['Buyer_Profit'] = 0
    df['Buyer_Loss'] = 0
    df['Seller_Profit'] = 0
    df['Seller_Loss'] = 0

    # Convert 'Time Stamp' to datetime for efficient comparison
    df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    # Use a list to collect results for better performance
    results = []

    # Iterate through each row to perform the logic
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc='Processing Rows'):
        # Calculate Buyer Profit and Loss
        if row['BuyersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['defence_target']

            if target_value > entry_value:
                buyer_profit = target_value - entry_value
                buyer_loss = 0
            else:
                buyer_profit = 0
                buyer_loss = abs(target_value - entry_value)
        else:
            buyer_profit = 0
            buyer_loss = 0

        # Calculate Seller Profit and Loss
        if row['SellersInControlPrice'] != 0:
            entry_value = row['Entry']
            target_value = row['defence_target']

            if entry_value > target_value:
                seller_profit = entry_value - target_value
                seller_loss = 0
            else:
                seller_profit = 0
                seller_loss = abs(entry_value - target_value)
        else:
            seller_profit = 0
            seller_loss = 0

        # Append results
        results.append({
            'index': index,
            'Buyer_Profit': buyer_profit,
            'Buyer_Loss': buyer_loss,
            'Seller_Profit': seller_profit,
            'Seller_Loss': seller_loss
        })

    # Update DataFrame with results
    results_df = pd.DataFrame(results)
    df.update(results_df.set_index('index'))
    df['D_Profit'] = df['Buyer_Profit'] + df['Seller_Profit']
    df['D_Loss'] = df['Buyer_Loss'] + df['Seller_Loss']
    df.drop(columns=["Buyer_Profit","Buyer_Loss","Seller_Profit","Seller_Loss"],axis=1,inplace=True)


    return df


In [23]:
def_data_1 = defence_process_data(trade_data_pro_loss)

Processing Rows: 100%|██████████| 505221/505221 [00:41<00:00, 12258.54it/s]


In [26]:
def_data_1.sample(10)

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,defence_target,Profit,Loss,D_Profit,D_Loss
268391,2022-02-25 08:35:00,4656.75,4667.50,4652.50,4666.00,4661.25,4660.25,4660.00,4659.25,4658.00,4640.50,4642.00,4644.25,4651.25,4652.00,02/25/2022 08:15,4629.75,0,0.00,4662.75,4644.25,4642.00,4664.0,4654.500,1.25,0,0.000,8.25
92371,2020-06-01 09:30:00,3322.00,3324.00,3321.75,3322.00,3323.25,3323.00,3323.00,3322.75,3322.25,3317.00,3317.25,3318.75,3321.75,3321.75,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00
244643,2021-12-03 03:35:00,4922.50,4922.50,4915.75,4916.75,4928.00,4927.75,4927.25,4927.00,4923.75,4919.75,4920.50,4920.75,4921.50,4922.25,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00
226630,2021-09-30 10:45:00,4701.50,4701.75,4693.25,4697.50,4714.75,4714.25,4713.25,4712.25,4707.00,4698.00,4698.50,4699.75,4699.75,4700.25,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00
475877,2024-03-06 22:45:00,5224.75,5226.75,5224.75,5226.50,5225.75,5225.75,5225.50,5225.50,5225.25,5222.00,5222.25,5222.75,5223.00,5223.00,03/06/2024 22:15,5224.75,0,0.00,5226.00,5225.75,5223.00,5226.5,5226.875,0.50,0,0.875,0.00
438643,2023-10-24 01:00:00,4435.25,4435.75,4435.00,4435.75,4436.00,4436.00,4436.00,4435.75,4435.50,4433.00,4433.75,4434.25,4434.75,4434.75,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00
71915,2020-03-18 01:10:00,2688.75,2692.00,2687.00,2691.75,2700.25,2699.00,2698.25,2697.50,2694.25,2686.75,2686.75,2687.50,2687.75,2688.00,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00
299513,2022-06-15 08:50:00,4133.75,4141.00,4132.75,4140.00,4137.25,4135.50,4134.75,4134.75,4134.00,4125.00,4125.25,4125.25,4126.25,4133.50,06/15/2022 08:15,4123.75,0,0.00,4137.75,4126.25,4124.75,4138.5,4133.000,0.75,0,0.000,4.75
480196,2024-03-22 00:10:00,5368.00,5369.50,5368.00,5369.00,5369.75,5369.50,5369.25,5368.75,5368.25,5367.00,5367.00,5367.75,5367.75,5367.75,0,0.00,03/21/2024 23:15,5366.25,5368.25,5368.75,5369.50,5368.0,5367.500,0.25,0,0.750,0.00
207872,2021-07-26 03:25:00,4726.50,4727.25,4725.00,4725.00,4728.75,4728.50,4728.25,4728.25,4728.25,4724.50,4724.75,4724.75,4725.25,4725.50,0,0.00,0,0.00,0.00,0.00,0.00,0.0,0.000,0.00,0,0.000,0.00


In [24]:
def_data_2 = defence_process_data(cal_data_pro_loss)

Processing Rows: 100%|██████████| 505221/505221 [00:40<00:00, 12446.23it/s]


In [25]:
def_data_3 = defence_process_data(ideal_data_pro_loss)

Processing Rows: 100%|██████████| 505221/505221 [00:50<00:00, 10016.93it/s]


In [28]:
import pandas as pd

def rearrange_columns(df):
    desired_order = [
        'Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
        'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
        'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
        'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
        'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence',
        'Stop', 'Target', 'Profit', 'Loss', 'defence_target', 'D_Profit',
        'D_Loss'
    ]

    # Ensure that only columns present in both the desired order and DataFrame are rearranged
    columns_in_order = [col for col in desired_order if col in df.columns]

    # Add any remaining columns not specified in the desired order
    remaining_columns = [col for col in df.columns if col not in desired_order]

    # Combine the lists to create the final order
    final_order = columns_in_order + remaining_columns

    # Reorder the DataFrame columns
    return df[final_order]

In [31]:
Trade_data_defence_pro_loss = rearrange_columns(def_data_1)
Cal_data_defence_pro_loss = rearrange_columns(def_data_2)
Ideal_data_defence_pro_loss = rearrange_columns(def_data_3)

In [38]:
Cal_data_defence_pro_loss.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry_C', 'Defence_C',
       'Stop_C', 'Target_C', 'Profit_C', 'Loss_C', 'defence_target_C',
       'D_Profit_C', 'D_Loss_C'],
      dtype='object')

In [35]:
s = "_T"
Trade_data_defence_pro_loss.columns = [col + s if col in ['Entry', 'Defence','Stop', 'Target', 'Profit', 'Loss', 'defence_target','D_Profit','D_Loss'] else col for col in Trade_data_defence_pro_loss.columns]
s = "_C"
Cal_data_defence_pro_loss.columns = [col + s if col in ['Entry', 'Defence','Stop', 'Target', 'Profit', 'Loss', 'defence_target','D_Profit','D_Loss'] else col for col in Cal_data_defence_pro_loss.columns]
s = "_Ideal"
Ideal_data_defence_pro_loss.columns = [col + s if col in [ 'Target', 'Profit', 'Loss', 'defence_target','D_Profit','D_Loss'] else col for col in Ideal_data_defence_pro_loss.columns]


In [40]:
Trade_data_defence_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/trade_data_defence_profit_loss_data.csv")
Cal_data_defence_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/cal_data_defence_profit_loss_data.csv")
Ideal_data_defence_pro_loss.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/ES_5min_data/Machine_Learning_Model/Defence_Profit_Loss_calculated/ideal_data_defence_profit_loss_data.csv")

### **Sample data for Visualization in chart **

In [ ]:

import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/')
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data/Target_Optimization_data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder, '/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data')  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Copy of ideal_target.csv',
 'Copy of New_5min_data (2).csv',
 'Copy of ES 09-24_Minute_5.csv',
 'df_target_comparision.csv',
 '.ipynb_checkpoints',
 'trade_data_profit_loss_data.csv',
 'cal_data_profit_loss_data.csv',
 'ideal_data_profit_loss_data.csv']

In [ ]:
#importent libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_columns',40)

In [ ]:
trade_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/trade_data_profit_loss_data.csv")  #trade data
cal_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/cal_data_profit_loss_data.csv")  #cal data
ideal_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/ideal_data_profit_loss_data.csv")  #ideal

In [ ]:
trade_data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
#trade_data['Time_Stamp'] = pd.to_datetime(trade_data['Time_Stamp'])
# cal_data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
# cal_data['Time Stamp'] = pd.to_datetime(cal_data['Time Stamp'])
# ideal_data.drop(columns=["Unnamed: 0"],axis=1,inplace=True)
# ideal_data['Time Stamp'] = pd.to_datetime(ideal_data['Time Stamp'])
trade_data.columns = trade_data.columns.str.strip()


In [ ]:
trade_data.dtypes
trade_data['Time Stamp'] = pd.to_datetime(trade_data['Time Stamp'])

In [ ]:
trade_data

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,Highest_value,Highest_timestamp,Stop_Match_timestamp,Profit,Loss
0,2019-06-24 00:00:00,3242.75,3243.50,3242.75,3243.50,0.00,0.00,0.00,0.00,3243.25,3240.50,3241.25,3241.25,3241.5,3241.50,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
1,2019-06-24 00:05:00,3243.25,3243.25,3242.50,3242.75,0.00,0.00,0.00,0.00,3244.25,3241.25,3241.50,3241.50,3242.0,3242.75,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
2,2019-06-24 00:10:00,3242.75,3243.00,3242.75,3243.00,0.00,0.00,0.00,3244.25,3243.25,3241.25,3241.25,3241.50,3241.5,3242.00,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
3,2019-06-24 00:15:00,3242.75,3243.00,3242.50,3242.75,0.00,0.00,3244.25,3243.50,3243.25,3241.25,3241.25,3241.50,3241.5,3242.00,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
4,2019-06-24 00:20:00,3242.75,3243.00,3242.75,3243.00,0.00,3244.25,3243.50,3243.25,3243.25,3241.25,3241.25,3241.50,3241.5,3242.00,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,2019-07-24 23:40:00,3304.50,3305.00,3304.50,3304.75,3305.25,3305.00,3305.00,3304.75,3304.75,3303.75,3304.00,3304.00,3304.0,3304.00,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
8307,2019-07-24 23:45:00,3304.75,3305.00,3304.50,3304.75,3305.25,3305.25,3305.25,3305.00,3305.00,3303.75,3304.00,3304.00,3304.0,3304.00,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
8308,2019-07-24 23:50:00,3304.75,3305.25,3304.75,3304.75,3305.25,3305.25,3305.25,3305.00,3305.00,3304.00,3304.00,3304.00,3304.0,3304.25,0,0.0,0,0.0,0.00,0.00,0.0,0.00,0.0,0,0,0.0,0.0
8309,2019-07-24 23:55:00,3304.75,3305.25,3304.75,3305.25,3305.25,3305.25,3305.25,3305.00,3305.00,3304.00,3304.00,3304.00,3304.0,3304.25,07/24/2019 22:45,3303.5,0,0.0,3305.00,3304.25,3304.0,3305.25,0.0,0,0,0.0,0.0


In [ ]:
start_time = '2019-06-24 13:15:00'
end_time = '2019-06-24 15:40:00'
mask = (trade_data['Time Stamp'] >= start_time) & (trade_data['Time Stamp'] <= end_time)
filtered_df = trade_data.loc[mask]

In [ ]:
#trade_data.set_index("Time Stamp",inplace=True)
# cal_data.set_index("Time Stamp",inplace=True)
# ideal_data.set_index("Time Stamp",inplace=True)

In [ ]:
filtered_df.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence',
       'Stop', 'Target', 'Highest_value', 'Highest_timestamp',
       'Stop_Match_timestamp', 'Profit', 'Loss'],
      dtype='object')

In [ ]:
filtered_df.shape

(35, 28)

In [ ]:
filtered_df

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,Highest_value,Highest_timestamp,Stop_Match_timestamp,Profit,Loss
186,2019-06-24 13:15:00,3237.75,3238.50,3237.50,3237.75,3239.00,3238.75,3238.25,3238.00,3238.00,3237.25,3237.50,3237.50,3237.50,3237.50,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.0
187,2019-06-24 13:20:00,3237.50,3238.50,3237.50,3238.25,3238.25,3238.00,3238.00,3237.75,3237.75,3237.25,3237.25,3237.25,3237.25,3237.25,06/24/2019 11:00,3237.25,0,0.00,3238.00,3237.75,3237.75,3238.75,3238.25,2019-06-24 13:25:00,2019-06-24 13:25:00,0.00,-0.5
188,2019-06-24 13:20:00,3237.50,3238.50,3237.50,3238.25,3238.25,3238.00,3238.00,3237.75,3237.75,3237.25,3237.25,3237.25,3237.25,3237.25,06/24/2019 11:00,3237.25,0,0.00,3238.00,3237.75,3237.75,3238.75,3238.25,2019-06-24 13:25:00,2019-06-24 13:25:00,0.00,-0.5
189,2019-06-24 13:25:00,3238.25,3238.25,3237.50,3237.75,3239.50,3239.50,3239.00,3238.75,3238.75,3237.50,3237.75,3237.75,3238.00,3238.00,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.0
190,2019-06-24 13:30:00,3237.75,3238.00,3237.25,3237.50,3239.50,3239.00,3238.75,3238.75,3238.25,3237.25,3237.50,3237.50,3237.50,3237.50,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.0
191,2019-06-24 13:35:00,3237.50,3238.00,3237.25,3237.50,3238.75,3238.75,3238.50,3238.25,3237.75,3237.25,3237.25,3237.25,3237.25,3237.25,0,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.0
192,2019-06-24 13:40:00,3237.75,3239.00,3237.50,3238.75,3238.75,3238.75,3238.50,3238.25,3238.25,3237.25,3237.50,3237.50,3237.50,3237.50,06/24/2019 11:00,3237.25,0,0.00,3238.25,3237.75,3237.50,3238.50,3239.00,2019-06-24 13:45:00,2019-06-24 20:05:00,0.50,0.0
193,2019-06-24 13:40:00,3237.75,3239.00,3237.50,3238.75,3238.75,3238.75,3238.50,3238.25,3238.25,3237.25,3237.50,3237.50,3237.50,3237.50,06/24/2019 11:00,3237.25,0,0.00,3237.75,3237.50,3237.50,3238.25,3239.00,2019-06-24 13:45:00,2019-06-24 20:05:00,0.75,0.0
194,2019-06-24 13:40:00,3237.75,3239.00,3237.50,3238.75,3238.75,3238.75,3238.50,3238.25,3238.25,3237.25,3237.50,3237.50,3237.50,3237.50,06/24/2019 11:00,3237.25,0,0.00,3238.50,3238.25,3237.75,3238.75,0.00,0,0,0.00,0.0
195,2019-06-24 13:40:00,3237.75,3239.00,3237.50,3238.75,3238.75,3238.75,3238.50,3238.25,3238.25,3237.25,3237.50,3237.50,3237.50,3237.50,06/24/2019 11:00,3237.25,0,0.00,3238.25,3237.75,3237.50,3238.50,3239.00,2019-06-24 13:45:00,2019-06-24 20:05:00,0.50,0.0


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_candlestick_with_control_prices(filtered_df):

    # Create a subplot with three rows
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=('BuyersInControlPrice', 'SellersInControlPrice', 'Profit/Loss'),
        shared_xaxes=True
    )

    # Add the candlestick chart to all subplots
    for i in range(1, 4):
        fig.add_trace(go.Candlestick(
            x=filtered_df['Time Stamp'],
            open=filtered_df['Open'],
            high=filtered_df['High'],
            low=filtered_df['Low'],
            close=filtered_df['Close'],
            name='Candlestick'
        ), row=i, col=1)

    # Find the first occurrence of BuyersInControlPrice and SellersInControlPrice
    first_buyer_control = filtered_df[filtered_df['BuyersInControlPrice'] != 0].iloc[0]
    first_seller_control = filtered_df[filtered_df['SellersInControlPrice'] != 0].iloc[0]

    # Add horizontal lines and annotations for the first BuyersInControlPrice
    if not first_buyer_control.empty:
        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_buyer_control['BuyersInControlPrice'], first_buyer_control['BuyersInControlPrice']],
            mode='lines',
            line=dict(color='green', width=2),
            hovertext=f'Entry: {first_buyer_control["BuyersInControlPrice"]}',
            hoverinfo='text',
            name='Entry',
            showlegend=False
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_buyer_control['Defence'], first_buyer_control['Defence']],
            mode='lines',
            line=dict(color='blue', width=2),
            hovertext=f'Defence: {first_buyer_control["Defence"]}',
            hoverinfo='text',
            name='Defence',
            showlegend=False
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_buyer_control['Stop'], first_buyer_control['Stop']],
            mode='lines',
            line=dict(color='purple', width=2),
            hovertext=f'Stop: {first_buyer_control["Stop"]}',
            hoverinfo='text',
            name='Stop',
            showlegend=False
        ), row=1, col=1)

        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_buyer_control['Target'], first_buyer_control['Target']],
            mode='lines',
            line=dict(color='red', width=2),
            hovertext=f'Target: {first_buyer_control["Target"]}',
            hoverinfo='text',
            name='Target',
            showlegend=False
        ), row=1, col=1)

    # Add horizontal lines and annotations for the first SellersInControlPrice
    if not first_seller_control.empty:
        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_seller_control['SellersInControlPrice'], first_seller_control['SellersInControlPrice']],
            mode='lines',
            line=dict(color='orange', width=2),
            hovertext=f'Entry: {first_seller_control["SellersInControlPrice"]}',
            hoverinfo='text',
            name='Entry',
            showlegend=False
        ), row=2, col=1)
        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_seller_control['Defence'], first_seller_control['Defence']],
            mode='lines',
            line=dict(color='cyan', width=2),
            hovertext=f'Defence: {first_seller_control["Defence"]}',
            hoverinfo='text',
            name='Defence',
            showlegend=False
        ), row=2, col=1)

        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_seller_control['Stop'], first_seller_control['Stop']],
            mode='lines',
            line=dict(color='magenta', width=2),
            hovertext=f'Stop: {first_seller_control["Stop"]}',
            hoverinfo='text',
            name='Stop',
            showlegend=False
        ), row=2, col=1)

        fig.add_trace(go.Scatter(
            x=[filtered_df['Time Stamp'].min(), filtered_df['Time Stamp'].max()],
            y=[first_seller_control['Target'], first_seller_control['Target']],
            mode='lines',
            line=dict(color='red', width=2),
            hovertext=f'Target: {first_seller_control["Target"]}',
            hoverinfo='text',
            name='Target',
            showlegend=False
        ), row=2, col=1)

    # Add markers for Profit and Loss in the third subplot
    for index, row in filtered_df.iterrows():
        if row['Profit'] > 0:
            marker_color = 'green'
            marker_text = f'Profit: {row["Profit"]}'
        else:
            marker_color = 'red'
            marker_text = f'Loss: {row["Loss"]}'

        fig.add_trace(go.Scatter(
            x=[row['Time Stamp']],
            y=[(row['High'] + row['Low']) / 2],
            mode='markers',
            marker=dict(color=marker_color, size=10),
            hovertext=marker_text,
            hoverinfo='text',
            name='Profit/Loss',
            showlegend=False
        ), row=3, col=1)
    # Add custom color legends on the left side of the chart
    annotations = [
        dict(x=-0.1, y=1.0, xref='paper', yref='paper', text='Entry (Buyers)', showarrow=False, font=dict(color='green')),
        dict(x=-0.1, y=0.95, xref='paper', yref='paper', text='Defence (Buyers)', showarrow=False, font=dict(color='blue')),
        dict(x=-0.1, y=0.9, xref='paper', yref='paper', text='Stop (Buyers)', showarrow=False, font=dict(color='purple')),
        dict(x=-0.1, y=0.85, xref='paper', yref='paper', text='Target (Buyers)', showarrow=False, font=dict(color='red')),
        dict(x=-0.1, y=0.75, xref='paper', yref='paper', text='Entry (Sellers)', showarrow=False, font=dict(color='orange')),
        dict(x=-0.1, y=0.7, xref='paper', yref='paper', text='Defence (Sellers)', showarrow=False, font=dict(color='cyan')),
        dict(x=-0.1, y=0.65, xref='paper', yref='paper', text='Stop (Sellers)', showarrow=False, font=dict(color='magenta')),
        dict(x=-0.1, y=0.6, xref='paper', yref='paper', text='Target (Sellers)', showarrow=False, font=dict(color='red')),


    ]

    fig.update_layout(
        title='Candlestick Chart with BuyersInControlPrice, SellersInControlPrice, and Profit/Loss',
        xaxis_title='Time',
        yaxis_title='Price',klj
        xaxis_rangeslider_visible=False,
        xaxis_tickformat='%Y-%m-%d %H:%M:%S',  # Format x-axis to show date and time
        xaxis_tickangle=-45,  # Rotate x-axis labels for better readability
        height=1200,  # Increase height to accommodate three subplots
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.02,
            xanchor="left",
            x=0
        ),
        annotations=annotations
    )

    return fig

# Example usage:
# Assuming filtered_df is your DataFrame with the required data
# fig = plot_candlestick_with_control_prices(filtered_df)
# fig.show()

In [ ]:
fig = plot_candlestick_with_control_prices(filtered_df)
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

